In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install Resemblyzer
!sudo apt update && sudo apt install ffmpeg
!pip install librosa
!pip install youtube_dl
!pip install datasets
!pip install pydub
!pip3 install spectralcluster
!pip install transformers

      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.5 MB/s eta 0:00:00
  Preparing metada

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [555 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,305 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 k

In [ ]:
import requests
import whisper
import time
import librosa
import soundfile as sf
import re
import os
import youtube_dl
import torch
from transformers import pipeline
from datasets import load_dataset
import numpy as np
import copy
import math
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from pydub import AudioSegment
from spectralcluster import SpectralClusterer
import resemblyzer
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation
from resemblyzer.audio import sampling_rate
from matplotlib import cm
from time import sleep, perf_counter as timer

from sys import stderr
import matplotlib.pyplot as plt
import numpy as np

# Video

In [ ]:
# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

# Initialize the device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-medium",
  chunk_length_s=30,
  device=device,
)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def seconds_to_time(seconds):
    if seconds is not None:
        hours = int(seconds / 3600)
        minutes = int((seconds % 3600) / 60)
        remaining_seconds = int(seconds % 60)
        hundredths = int((seconds - int(seconds)) * 100)
    else:
        hours = 0
        minutes = 0
        remaining_seconds = 0
        hundredths = 0
    return '{:02d}:{:02d}:{:02d}.{:02d}'.format(hours, minutes, remaining_seconds, hundredths)

In [ ]:
import copy
# Get the list of video files from the WhisperVideo folder
target_path = "/content/drive/MyDrive/WhisperVideo/"
video_files = os.listdir(target_path)

# Loop through the video files and transcribe them
for video_file in video_files:

    # Skip the file if it is not a video format
    if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
        continue

    print(f"Currently processing {video_file}")

    # Extract the audio from the video file using librosa
    video_path = target_path + video_file
    audio_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file[:-4] + ".wav" # Replace the video extension with .wav
    # if 'y' not in locals() and 'y' not in globals():
    if not os.path.isfile(audio_path):
        y, sr = librosa.load(video_path, sr=16000)
    else:
        y, sr = librosa.load(audio_path, sr=16000)

    print(y)
    print(type(y))

    if not os.path.isfile(audio_path):
        sf.write(audio_path, y, sr) # Save the audio as a wav file

    print(f"{video_file} audio file has been extracted {audio_path}")

    audio_sample = {
        'path': audio_path,
        'array': y,
        'sampling_rate': sr
    }

    trial = 0
    while True:
        try:
            trial = trial + 1
            # we can also return timestamps for the predictions
            prediction_chunk = pipe(audio_sample.copy(), return_timestamps=True)["chunks"]
            break
        except Exception as e:
            print(e)
            if trial >= 2:
                break
    if trial >= 2:
        print(f"{audio_path} cannot be processed")
        continue

    timestamp_text = []
    pure_text = ''
    for pc in prediction_chunk:
        time_start = seconds_to_time(pc['timestamp'][0])
        time_end = seconds_to_time(pc['timestamp'][1])
        data = {
            'text': pc['text'],
            'start_time': time_start,
            'end_time': time_end
        }
        timestamp_text.append(data)
        pure_text = pure_text + '\n' + pc['text']

    saved_timestamp_text = ''
    for ts_text in timestamp_text:
        saved_timestamp_text = saved_timestamp_text + f'\n[{ts_text["start_time"]}] [{ts_text["end_time"]}] {ts_text["text"]}\n\n'

    # Save the transcription as a text file in Google Docs
    text_file = video_file[:-4] + ".txt" # Replace the video extension with .txt
    timestamp_text_file = video_file[:-4] + "_timestamp_" + ".txt"
    text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
    timestamp_text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + timestamp_text_file
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(pure_text)
    with open(timestamp_text_path, "w", encoding="utf-8") as f:
        f.write(saved_timestamp_text)

    # Move the video file to the ProcessedVideo folder
    processed_path = "/content/drive/MyDrive/WhisperVideo/ProcessedVideo/" + video_file
    os.rename(video_path, processed_path)

    # Print a message to indicate the progress
    print(f"Processed {video_file} and saved the transcription as {text_file}")

# Audio